## Modules

In [1]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
from itertools import product
from pandas import ExcelWriter
from scipy.stats import norm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# `bsoption` module
from bsoption.bsmodel import *

# `hkexoi` module
from hkexoi import *
from hkexoi.hktradedates import *
from hkexoi.database import *

## Option strategy object

### strategy ratio dictionary

In [2]:
stratdict = {
        'spread': (1, -1),  # debit/credit spread: LONG 1 leg SHORT other
        'strangle': (1, 1),  # strangle / straddle: Same direction both side
        'synthetic': (1, -1),  # LC + SP / LP + SC
        }

### Option duo class

In [13]:
class Opduo():
    """
    Formulate option strategy involving 2 distinct option series.
    """

    def __init__(self, op1, str1, exp1, sig1, op2, str2, exp2, sig2, strat, firstside, q=0, rf=0):
        self.op1 = op1  # Option type of first instrument (call or put)
        self.str1 = str1  # 1st strike price
        self.exp1 = exp1  # 1st expiry date
        self.sig1 = sig1 # 1st IV
        self.op2 = op2  # Option type of first instrument (call or put)
        self.str2 = str2  # 2nd strike price
        self.exp2 = exp2  # 2nd expiry date
        self.sig2 = sig2  # 2nd IV
        self.strat = strat # option combo name
        self.firstside = firstside # LONG/SHORT for the 1st leg
        self.q = q  # Dividend rate
        self.rf = rf  # risk-free rate

    def getduomodel(self, spot, tradedate):
        """Obtain a pair of BSModel object given trading date, spot & IVs."""
        day1 = (self.exp1 - tradedate).days  # days to expiry of option 1
        day2 = (self.exp2 - tradedate).days  # days to expiry of option 2
        BS1 = BSModel(spot, self.str1, day1, self.sig1)  # BSModel of option 1
        BS2 = BSModel(spot, self.str2, day2, self.sig2)  # BSModel of option 2
        return BS1, BS2

    def getstratspec(self, spot, tradedate, digit=2):
        """Obtain strategy price & greeks. LONG/SHORT refers to 1st leg."""
        # Pair of BSModel objects
        BS1, BS2 = self.getduomodel(spot, tradedate)
        opprice1 = BS1.getopprice(self.op1)
        opprice2 = BS2.getopprice(self.op2)
        # identify option strategy type
        ratiopair = stratdict[self.strat]
        sign = lambda x: 1 if x == 'LONG' else -1
        side = self.firstside
        # Strategy price
        stratprice = round((opprice1 * ratiopair[0] + opprice2 * ratiopair[1]) * sign(side), digit)
        # Strategy delta
        delta = round((BS1.getdelta(self.op1) * ratiopair[0] + BS2.getdelta(self.op2) * ratiopair[1]) * sign(side), 4)
        # strategy theta
        theta = round((BS1.gettheta(self.op1) * ratiopair[0] + BS2.gettheta(self.op2) * ratiopair[1]) * sign(side), 4)
        # strategy vega
        vega = round((BS1.vega * ratiopair[0] + BS2.vega * ratiopair[1]) * sign(side), 4)
        #  strategy gamma
        gamma = round((BS1.gamma * ratiopair[0] + BS2.gamma * ratiopair[1]) * sign(side), 4)

        return stratprice, delta, theta, vega, gamma
    
    def getstratpayoff(self, preexpiry=False):
        """
        Obtain payoff diagram at expiry and (if `preexpiry` enabled) payoff of each given days before expiry.
        Expiry for the two legs must be the same. (Thus calendar spread is NOT available for payoff diagram.)
        """
        halfplus = lambda x: x if x > 0 else 0
        sig = max([self.sig1, self.sig2])
        lowb = self.K * (1 - self.sig / 2)
        upb = self.K * (1 + self.sig / 2)
        pricearr = np.linspace(lowb, upb, 11)
        
        
        
        return


### Example 1: Debit call spread

In [4]:
# Spec of 1st leg (LC)
op11 = 'C'
str11 = 3200
exp11 = datetime(2022, 4, 29)

# Spec of 2nd 
op21 = 'C'
str21 = 4000
exp21 = exp11

# Current underlying price & date
spot1 = 2800
td1 = datetime(2022, 3, 21)

# Strat nature
strat1 = 'spread'
firstside1 = 'LONG'

# For each IV level, compute debit call spread price
sigrange = range(32, 144, 16)
for sig in sigrange:
    DCspread = Opduo(op11, str11, exp11, sig / 100, op21, str21, exp21, sig / 100, strat1, firstside1)
    price, delta, theta, vega, gamma = DCspread.getstratspec(spot1, td1) 
    print(f'IV {sig}%: price={price}, delta={delta}, theta={theta}, vega={vega}, gamma={gamma}')
    

IV 32%: price=14.94, delta=0.11, theta=-0.7027, vega=1.7127, gamma=0.0006
IV 48%: price=49.38, delta=0.2058, theta=-1.4651, vega=2.3807, gamma=0.0006
IV 64%: price=86.3, delta=0.242, theta=-1.7658, vega=2.152, gamma=0.0004
IV 80%: price=117.09, delta=0.2433, theta=-1.7335, vega=1.6902, gamma=0.0003
IV 96%: price=140.61, delta=0.2305, theta=-1.5537, vega=1.2624, gamma=0.0002
IV 112%: price=157.96, delta=0.2135, theta=-1.3232, vega=0.9215, gamma=0.0001
IV 128%: price=170.52, delta=0.1962, theta=-1.0838, vega=0.6604, gamma=0.0001


### Example 2: Calendar spread (same strike, different expiry)

In [5]:
# Spec of 1st leg (SC)
op12 = 'C'
str12 = 3200
exp12 = datetime(2022, 4, 29)

# Spec of 2nd (LC)
op22 = 'C'
str22 = str12
exp22 = datetime(2022, 5, 27)

# Current underlying price & date
spot2 = 2800
td2 = datetime(2022, 3, 21)

# Strat nature
strat2 = 'spread'
firstside2 = 'SHORT'

# For each IV level, compute debit call spread price
sigrange = range(32, 144, 16)
for sig in sigrange:
    CLspread = Opduo(op12, str12, exp12, sig / 100, op22, str22, exp22, sig / 100, strat2, firstside2)
    price, delta, theta, vega, gamma = CLspread.getstratspec(spot2, td2) 
    print(f'IV {sig}%: price={price}, delta={delta}, theta={theta}, vega={vega}, gamma={gamma}')

IV 32%: price=20.9, delta=0.0722, theta=-0.0506, vega=1.4507, gamma=0.0
IV 48%: price=44.02, delta=0.0725, theta=0.1906, vega=1.4128, gamma=-0.0001
IV 64%: price=65.98, delta=0.0664, theta=0.4482, vega=1.335, gamma=-0.0001
IV 80%: price=86.85, delta=0.0616, theta=0.6943, vega=1.2761, gamma=-0.0001
IV 96%: price=106.88, delta=0.0585, theta=0.9302, vega=1.2301, gamma=-0.0001
IV 112%: price=126.24, delta=0.0568, theta=1.16, vega=1.1907, gamma=-0.0001
IV 128%: price=145.0, delta=0.056, theta=1.3865, vega=1.1542, gamma=-0.0001


### Example 3: Synthetic SF (LP + SC)

In [7]:
# Spec of 1st leg (LP)
op13 = 'P'
str13 = 7800
exp13 = datetime(2022, 3, 30)
sig13 = 32

# Spec of 2nd leg (SC)
op23 = 'C'
str23 = 8200
exp23 = exp13
sig23 = sig13

# Current underlying date
td3 = datetime(2022, 2, 28)

# Strat nature
strat3 = 'synthetic'
firstside3 = 'LONG'

# For each IV level, compute debit call spread price
spotrange = range(5600, 10400, 400)
for spot in spotrange:
    SynSF = Opduo(op13, str13, exp13, sig13 / 100, op23, str23, exp23, sig23 / 100, strat3, firstside3)
    price, delta, theta, vega, gamma = SynSF.getstratspec(spot, td3) 
    print(f'Spot {spot}: price={price}, delta={delta}, theta={theta}, vega={vega}, gamma={gamma}')
    

Spot 5600: price=2200.02, delta=-0.9998, theta=-0.0052, vega=0.0097, gamma=0.0
Spot 6000: price=1800.34, delta=-0.9979, theta=-0.0568, vega=0.1066, gamma=0.0
Spot 6400: price=1402.88, delta=-0.9865, theta=-0.3062, vega=0.5741, gamma=0.0001
Spot 6800: price=1014.56, delta=-0.9495, theta=-0.8832, vega=1.656, gamma=0.0001
Spot 7200: price=647.71, delta=-0.8808, theta=-1.4067, vega=2.6376, gamma=0.0002
Spot 7600: price=310.17, delta=-0.8108, theta=-1.0937, vega=2.0506, gamma=0.0001
Spot 8000: price=-7.05, delta=-0.7854, theta=0.1262, vega=-0.2365, gamma=-0.0
Spot 8400: price=-326.09, delta=-0.8178, theta=1.3265, vega=-2.4872, gamma=-0.0001
Spot 8800: price=-665.22, delta=-0.8794, theta=1.7222, vega=-3.2292, gamma=-0.0002
Spot 9200: price=-1028.82, delta=-0.9357, theta=1.3875, vega=-2.6016, gamma=-0.0001
Spot 9600: price=-1411.0, delta=-0.9716, theta=0.8284, vega=-1.5533, gamma=-0.0001
Spot 10000: price=-1803.67, delta=-0.9894, theta=0.3943, vega=-0.7392, gamma=-0.0


### Example 4: Strangle (IF same strike then becomes straddle)

In [14]:
# Spec of 1st leg (LP)
op14 = 'P'
str14 = 7800
exp14 = datetime(2022, 3, 30)
sig14 = 24

# Spec of 2nd leg (LC)
op24 = 'C'
str24 = 8200
exp24 = exp14
sig24 = sig14

# Current underlying date
td4 = datetime(2022, 2, 28)

# Strat nature
strat4 = 'strangle'
firstside4 = 'LONG'

# For each IV level, compute debit call spread price
spotrange = range(5600, 10400, 400)
for spot in spotrange:
    Strangle = Opduo(op14, str14, exp14, sig14 / 100, op24, str24, exp24, sig24 / 100, strat4, firstside4)
    price, delta, theta, vega, gamma = Strangle.getstratspec(spot, td4) 
    print(f'Spot {spot}: price={price}, delta={delta}, theta={theta}, vega={vega}, gamma={gamma}')
    

Spot 5600: price=2200.0, delta=-1.0, theta=-0.0, vega=0.0001, gamma=0.0
Spot 6000: price=1800.01, delta=-0.9999, theta=-0.0023, vega=0.0057, gamma=0.0
Spot 6400: price=1400.31, delta=-0.9976, theta=-0.0568, vega=0.1421, gamma=0.0
Spot 6800: price=1004.83, delta=-0.9714, theta=-0.5403, vega=1.3508, gamma=0.0001
Spot 7200: price=637.15, delta=-0.8388, theta=-2.3304, vega=5.8261, gamma=0.0006
Spot 7600: price=363.13, delta=-0.4919, theta=-5.2398, vega=13.0995, gamma=0.0011
Spot 8000: price=267.77, delta=0.0291, theta=-6.8476, vega=17.119, gamma=0.0014
Spot 8400: price=381.58, delta=0.5166, theta=-5.641, vega=14.1026, gamma=0.001
Spot 8800: price=655.43, delta=0.8187, theta=-3.1084, vega=7.7711, gamma=0.0005
Spot 9200: price=1013.27, delta=0.9486, theta=-1.1987, vega=2.9967, gamma=0.0002
Spot 9600: price=1402.51, delta=0.9888, theta=-0.3358, vega=0.8394, gamma=0.0
Spot 10000: price=1800.38, delta=0.9981, theta=-0.0706, vega=0.1765, gamma=0.0


## Payoff diagram

## Incorporate with `hkexoi` module

In [ ]:
def gethkstratdf(symbol, strat, optype1, month1, strike1, optype2, month2, strike2, startstr, endstr, 
                 side='LONG', style='regular'):
    """Obtain the option strat price and greeks."""
    assert style in ['regular', 'settle'], AttributeError('Inappropriate session style!')
    assert side in ['LONG', 'SHORT'], AttributeError('Must be LONG or SHORT!')
    sign = lambda x: 1 if x == 'LONG' else -1  
    dfop1 = get_oneop(symbol, optype1, month1, strike1, startstr, endstr, style)
    dfop2 = get_oneop(symbol, optype2, month2, strike2, startstr, endstr, style)
    if style == 'regular':
        closefield = 'td_close'
    else:
        closefield = 'day_close'
    # Strategy dataframe columns
    opfields = ['month', 'strike', 'optype', 'sig', 'ftclose']
    specfields = [closefield, 'delta', 'theta', 'vega']
    stratcol = [f'{field}{num}' for field, num in product(opfields, range(1, 3))] + specfields
    dfstrat = pd.DataFrame()
    for field in opfields:
        dfstrat[f'{field}1'] = dfop1[field]
        dfstrat[f'{field}2'] = dfop2[field]
    ratiopair = stratdict[strat]
    for field in specfields:
        dfstrat[field] = (dfop1[field] * ratiopair[0] + dfop2[field] * ratiopair[1]) * sign(side)
    
    return dfstrat

### Sample : call spread

In [ ]:
strattest = 'spread'
op1test = 'C'
str1test = 29000
month1test = (2021, 4)
op2test = 'C'
str2test = 30000
month2test = month1test
startstr = '2021-04-01'
endstr = '2021-04-30'
dfstrattest = gethkstratdf('hsi', strattest, op1test, month1test, str1test, op2test, month2test, str2test, startstr, endstr)
dfstrattest

### Sample: strangle

In [ ]:
strattest = 'strangle'
op1test = 'C'
str1test = 29000
month1test = (2021, 4)
op2test = 'P'
str2test = 28000
month2test = month1test
startstr = '2021-04-01'
endstr = '2021-04-30'
dfstrattest2 = gethkstratdf('hsi', strattest, op1test, month1test, str1test, op2test, month2test, str2test, startstr, endstr)
dfstrattest2

### Sample: Calendar spread

In [ ]:
strattest = 'spread'
op1test = 'C'
str1test = 28400
month1test = (2021, 2)
op2test = 'C'
str2test = 28400
month2test = (2021, 1)
startstr = '2021-01-01'
endstr = '2021-01-28'
dfstrattest4 = gethkstratdf('hsi', strattest, op1test, month1test, str1test, op2test, month2test, str2test, 
                            startstr, endstr)
dfstrattest4

## Option equity curve

### Function

In [ ]:
def getopstratcurve(dfop, style='regular'):
    """Obtain option strategy price curve."""
    assert style in ['regular', 'settle'], AttributeError('Inappropriate session style!')
    if style == 'regular':
        closefield = 'td_close'
    else:
        closefield = 'day_close'
    fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.04, 
                        row_heights=[0.32, 0.32, 0.12, 0.12, 0.12], specs=[[{"type": "scatter"}]] * 5,
                        subplot_titles=("Strategy", "Underlying", "Delta", "Theta", "Vega"))
    # Option price curve
    fig.add_trace(go.Scatter(x=dfop.index, y=dfop[closefield],
                  mode="lines", name="Strategy", line_color = '#43b117'), row=1, col=1)
    # Underlying Futures price curve
    fig.add_trace(go.Scatter(x=dfop.index, y=dfop['ftclose1'],
                  mode="lines", name="Underlying", line_color = '#c557d2'), row=2, col=1)
    fig.add_trace(go.Scatter(x=dfop.index, y=dfop['ftclose2'],
                  mode="lines", name="Underlying", line_color = '#1756b1'), row=2, col=1)
    # Delta curve
    fig.add_trace(go.Scatter(x=dfop.index, y=dfop['delta'],
                  mode="lines", name="Delta", line_color = '#b119b1'), row=3, col=1)
    # Theta curve
    fig.add_trace(go.Scatter(x=dfop.index, y=dfop['theta'],
                  mode="lines", name="Theta", line_color = '#c50c47'), row=4, col=1)
    # Vega curve
    fig.add_trace(go.Scatter(x=dfop.index, y=dfop['vega'],
                  mode="lines", name="Vega", line_color = '#34d0dd'), row=5, col=1)    
    # Chart title
    fig.update_layout(height=800, showlegend=False, title_text="Option Strategy", title_x=0.5)
    fig.show()

### Sample: spread

In [ ]:
strattest = 'spread'
op1test = 'C'
str1test = 29000
month1test = (2021, 4)
op2test = 'C'
str2test = 30000
month2test = month1test
startstr = '2021-04-01'
endstr = '2021-04-30'
dfstrattest1 = gethkstratdf('hsi', strattest, op1test, month1test, str1test, op2test, month2test, str2test, startstr, endstr)
getopstratcurve(dfstrattest1)

### Sample: Calendar spread

In [ ]:
getopstratcurve(dfstrattest4)

### Sample: strangle

In [ ]:
strattest = 'strangle'
op1test = 'C'
str1test = 29000
month1test = (2021, 4)
op2test = 'P'
str2test = 28000
month2test = month1test
startstr = '2021-04-01'
endstr = '2021-04-30'
dfstrattest2 = gethkstratdf('hsi', strattest, op1test, month1test, str1test, op2test, month2test, str2test, startstr, endstr)
getopstratcurve(dfstrattest2)